<a href="https://colab.research.google.com/github/jupppang/edu_jeonju_test/blob/main/06_13_LLaMAIndexParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LlamaParser
LlamaParse는 LlamaIndex에서 개발한 문서 파싱 서비스로, 대규모 언어 모델(LLM)을 위해 특별히 설계되었습니다. 주요 특징은 다음과 같습니다

- PDF, Word, PowerPoint, Excel 등 다양한 문서 형식 지원
- 자연어 지시를 통한 맞춤형 출력 형식 제공
- 복잡한 표와 이미지 추출 기능
- JSON 모드 지원
- 외국어 지원

LlamaParse는 독립형 API로 제공되며, LlamaCloud 플랫폼의 일부로도 사용 가능합니다. 이 서비스는 문서를 파싱하고 정제하여 검색 증강 생성(RAG) 등 LLM 기반 애플리케이션의 성능을 향상시키는 것을 목표로 합니다.

사용자는 무료로 하루 1,000페이지를 처리할 수 있으며, 유료 플랜을 통해 추가 용량을 확보할 수 있습니다. LlamaParse는 현재 공개 베타 버전으로 제공되고 있으며, 지속적으로 기능이 확장되고 있습니다.

In [11]:
!pip install -qU llama-index-core llama-parse llama-index-readers-file python-dotenv
!pip install -qU langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.0 MB/s eta 0:00:00


In [36]:
import os
import nest_asyncio

LLAMA_CLOUD_API_KEY = ''
nest_asyncio.apply()

In [37]:
os.environ['LLAMA_CLOUD_API_KEY'] = LLAMA_CLOUD_API_KEY

## Basic Parser

In [5]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# 파서 설정
parser = LlamaParse(
    result_type="markdown",  # "markdown"과 "text" 사용 가능
    num_workers=8,  # worker 수 (기본값: 4)
    verbose=True,
    language="ko",
    api_key=LLAMA_CLOUD_API_KEY,
)

# SimpleDirectoryReader를 사용하여 파일 파싱
file_extractor = {".pdf": parser}

# LlamaParse로 파일 파싱
documents = SimpleDirectoryReader(
    input_files=["/content/data/SPRI_AI_Brief_2023년12월호_F.pdf"],
    file_extractor=file_extractor,
).load_data()


Started parsing the file under job_id 9419670b-6fbd-4508-a110-fc67b9cc4748


In [6]:
len(documents)

23

In [8]:
documents[0]

Document(id_='6bbda00e-3079-461c-98b2-7e55f91be6a7', embedding=None, metadata={'file_path': '/content/data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_name': 'SPRI_AI_Brief_2023년12월호_F.pdf', 'file_type': 'application/pdf', 'file_size': 975735, 'creation_date': '2025-01-07', 'last_modified_date': '2025-01-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='# 인공지능 산업의 최신 동향\n\n2023년 12월호', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}')

### llamaindex -> langchain doument로 변환

In [12]:
# 랭체인 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in documents]

In [16]:
print(docs[10].page_content)

# SPRi Brief AI

# 2023-12월호

# 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개

코히어와 12개 기관이 광범위한 데이터셋에 감사를 통해 원본 데이터 출처, 재라이선스 상태, 작성자 등 다양한 정보를 제공하는 ‘데이터 출처 탐색기’ 플랫폼을 출시

대화형 플랫폼을 통해 개발자들이 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며 데이터셋의 구성과 계보도 추적 가능

# 데이터 출처 탐색기, 광범위한 데이터셋 정보 제공을 통해 데이터 투명성 향상

AI 기업 코히어(Cohere)가 매사추세츠공과대(MIT), 하버드대 로스쿨, 카네기멜론대 등 12개 기관과 함께 2023년 10월 25일 ‘데이터 출처 탐색기(Data Provenance Explorer)’ 플랫폼을 공개

- AI 모델 훈련에 사용되는데 데이터셋의 불분명한 출처로 인해 데이터 투명성이 확보되지 않아 다양한 법적·윤리적 문제가 발생
- 이에 연구진은 가장 널리 사용되는 2,000여 개의 미세 조정 데이터셋을 감사 및 추적하여 데이터셋에 원본 데이터 소스에 대한 태그, 재라이선스(Relicensing) 상태, 작성자, 기타 데이터 속성을 지정하고 이러한 정보에 접근할 수 있는 플랫폼을 출시
- 대화형 플랫폼 형태의 데이터 출처 탐색기를 통해 데이터셋의 라이선스 상태를 쉽게 파악할 수 있으며, 주요 데이터셋의 구성과 데이터 계보도 추적 가능

# 연구진은 오픈소스 데이터셋에 대한 광범위한 감사를 통해 데이터 투명성에 영향을 미치는 주요 요인을 발견

- 깃허브(GitHub), 페이퍼위드코드(Papers with Code)와 같은 크라우드소싱 플랫폼에서 수집한 데이터로 훈련된 오픈소스 LLM에서 데이터 라이선스의 누락 비율이 72~83%에 달함
- 또한 크라우드소싱 플랫폼이 할당한 라이선스는 데이터셋 원저작자의 의도보다 더 광범위한 사용을 허용한 경우가 상당수
- 데이터 생태계 분석 결과, 부정확하거나 모호한 라이선스 문서화 등 데이터 출처 입증과 관련된 관행 전반에서

In [13]:
docs[0].metadata

{'file_path': '/content/data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_name': 'SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_type': 'application/pdf',
 'file_size': 975735,
 'creation_date': '2025-01-07',
 'last_modified_date': '2025-01-07'}

## MultiModal Model
- use_vendor_multimodal_model: 멀티모달 모델 사용 여부를 지정합니다. True로 설정하면 외부 벤더의 멀티모달 모델을 사용합니다.

- vendor_multimodal_model_name: 사용할 멀티모달 모델의 이름을 지정합니다. 여기서는 "openai-gpt4o"를 사용하고 있습니다.

- vendor_multimodal_api_key: 멀티모달 모델 API 키를 지정합니다. 환경 변수에서 OpenAI API 키를 가져옵니다.

- result_type: 파싱 결과의 형식을 지정합니다. "markdown"으로 설정되어 있어 결과가 마크다운 형식으로 반환됩니다.

- language: 파싱할 문서의 언어를 지정합니다. "ko"로 설정되어 한국어로 처리됩니다.

- skip_diagonal_text: 대각선 텍스트를 건너뛸지 여부를 결정합니다.

- page_separator: 페이지 구분자를 지정할 수 있습니다

In [30]:
OPENAI_API_KEY = ""

In [26]:
documents = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.getenv("OPENAI_API_KEY"),
    result_type="markdown",
    language="ko",
    api_key=LLAMA_CLOUD_API_KEY,
    #skip_diagonal_text=True,
    page_separator="\n=================\n",
)

In [27]:
#pasing 된 결
parsed_docs = documents.load_data(file_path="/content/data/SPRI_AI_Brief_2023년12월호_F.pdf")

Started parsing the file under job_id 0bfc16d9-32ae-46bc-a3fe-d151d4f532b1


In [24]:
# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

In [28]:
print(docs[18].page_content)

# 구글 딥마인드, 범용 AI 모델의 가능과 동작에 대한 분류 체계 발표

## KEY Contents

- 구글 딥마인드 연구진이 성능과 범용성, 자율성을 기준으로 범용 AI(AGI)의 수준을 0-5단계까지 총 6단계로 구분한 프레임워크를 공개
- 현재 AGI는 단백질 구조를 예측하는 알파폴드와 같은 특정 용도에서는 5단계 수준을 달성했지만 광범위하게 활용될 수 있는 범용에서는 1단계 수준에 머물러 있음

## 챗GPT와 구글 바드와 같은 AI 챗봇은 범용 AI 1단계 수준

- 구글 딥마인드 연구진은 2023년 11월 4일 범용 AI(Artificial General Intelligence, AGI) 모델을 용도와 성능에 따라 분류하는 프레임워크를 제시한 논문을 발표
- 프레임워크의 목적은 AGI의 성능, 범용성, 자율성 수준을 정의하여 모델 간 비교와 위험 평가, AGI 달성까지의 진행 상황을 추정할 수 있는 공통 기준을 제공하기 위함
- 연구진은 AGI 개념 정리에 필요한 기준을 수립하기 위한 6가지 원칙을 아래와 같이 도출
  - (프로세스) 어떤 기능에 중점) AI가 어떻게 작동하는지보다 무엇을 할 수 있는지가 더 중요
  - (범용성 성능으로 모두 평가) 진정한 AGI는 인간을 능가하는 폭넓은 범용성과 기술의 리을 모두 요구
  - (인지와 메타인지 작업에 중점) 물리적 작업의 수행 능력은 AGI의 필수 전제조건이 아니며, 인지 작업과 메타인지 작업(예: 새로운 작업의 학습 능력, 인간에게 도움을 요청할 시점을 아는 능력)이 핵심
  - (실제 구현된 장치레에 집중) 통제된 상황에서 발휘되는 성능에 따라 AGI를 규정하고 테스트를 진행
  - (생태학적 타당도를 갖춘 벤치마크 사용) AGI에 대한 벤치마크는 사람들이 경제적·사회적 또는 예술적으로 가치 있게 여기는 실질적인 작업을 대상으로 성능 평가 필요
  - (충족이 아닌 AGI를 향한 경로에 중점) 단계별 접근방식을 통해 AGI의 발전 상태를 점진적으로 측정

- 연구진은 상기 원칙에 따라 A

## parsing instruction

In [33]:
OPENAI_API_KEY = ""

In [41]:
# parsing instruction 을 지정합니다.
parsing_instruction = (
    "You are parsing a brief of AI Report. Please extract tables in markdown format."
)

# LlamaParse 설정
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.getenv("OPENAI_API_KEY"),
    result_type="markdown",
    language="ko",
    api_key=LLAMA_CLOUD_API_KEY,
    parsing_instruction=parsing_instruction,
)

# parsing 된 결과
parsed_docs = parser.load_data(file_path="/content/data/SPRI_AI_Brief_2023년12월호_F.pdf")

# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]


Started parsing the file under job_id 313119b6-2613-44dd-87db-349e7a961836


In [42]:
# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

In [43]:
print(docs[18].page_content)

I'm unable to provide a direct translation of the text in the image, but I can help summarize the content and extract the table for you.

### Summary
The document discusses a framework for categorizing the capabilities and levels of Artificial General Intelligence (AGI) based on performance, scope, and autonomy. It introduces a six-level framework ranging from 0 to 5, with current AI models like ChatGPT and Google's Bard classified at level 1. The framework aims to provide common criteria for evaluating AGI's progress and potential applications.

### Table: 구글 딥마인드의 범용 AI 분류 프레임워크

| 성능  | 특수 AI 예시 | 범용 AI 예시 |
|-------|--------------|--------------|
| 0단계: AI 아님 | 계산기, 스프레드시트, 웹페이지 | 아마존 메커니컬 터크 |
| 1단계: 신진(초보적인) 인공 인간 | GOFAI(고전적 인공지능) | 챗GPT, 바드, 라마 |
| 2단계: 유숙(숙련된 인간의 50% 이상) | 스마트 스피커(예: 시리, 아마존 알렉사), 구글 어시스턴트, IBM 왓슨 | 미달성 |
| 3단계: 전문가(숙련된 인간의 90% 이상) | 로봇 조작/제어, 생성 이미지(프롬프트) | 미달성 |
| 4단계: 최적(숙련된 인간의 99% 이상) | 팀봇들, 알파폴드 | 미달성 |
| 5단계: 초인간(인간 100% 능가) | 알파폴드, 알파제로, 스톡피시 | 미달성 |

